# 03 — Adversarial Fine-tuning (demo)
This notebook runs **adversarial** fine-tuning on the tiny synthetic dataset.

**We call the training scripts by file path** (no `python -m`) to avoid import issues.

> ⚠️ Demo only — tiny epochs/trials for speed; results are not meaningful.

In [ ]:
# Optional: install the package into this kernel (usually not required if you use file paths)
# %pip install -e ../../mz_embed

In [ ]:
import os, shutil, pandas as pd, numpy as np
DATA = "../data/synthetic_small"

# Ensure a nuisance column for adversarial learning
for split in ("Train","Val","Test","Discovery_Train","Discovery_Val"):
    ycand = [f"y_Finetune_{split}.csv", f"y_Pretrain_{split}.csv"]
    for yname in ycand:
        ypath = os.path.join(DATA, yname)
        if os.path.exists(ypath):
            df = pd.read_csv(ypath, index_col=0)
            if "Treatment" not in df.columns:
                rng = np.random.default_rng(0)
                df["Treatment"] = rng.choice(["A","B"], size=len(df))
                df.to_csv(ypath)
            break
print("Checked/added Treatment column where applicable.")

In [ ]:
%%bash
set -e
python ../../mz_embed/finetune/retrain_adverserial_cox_finetune_VAE_main.py \
  --input_data_location "../data/synthetic_small" \
  --finetune_save_dir "../../out_adv_finetune_demo" \
  --task "DEMO OS" \
  --task_event "OS_event" \
  --adv_task "Treatment" \
  --lambda_adv 0.1 \
  --num_epochs 3 --batch_size 32 --patience 0

In [ ]:
import os
out_dir = "../../out_adv_finetune_demo"
print("Artifacts in", out_dir, ":", os.listdir(out_dir) if os.path.exists(out_dir) else "(not found)")